In [93]:
import numpy as np
import matplotlib.pyplot as plt

from utils import restrict, interpolate, smooth

In [ ]:
# Load function
def f(t):
    return np.ones_like(t)

def P(i):
    """Generate the problem for 1-dimensional discrete Poisson equaton

    Parameters
    ----------
    i : int
        Number of unknowns will be 2**i - 1

    Returns
    -------
    numpy.array
        System matrix  
    numpy.array
        right-hand side
    numpy.array
        grid
    """
    number_of_points = 2**i + 1
    t = np.linspace(0, 1, number_of_points)
    T = utils.system_matrix_1d(i)
    b = f(t[1:-1])
    return T, b, t

def solve(i):
    T, b, _ = P(i)
    x = np.linalg.solve(T, b)
    return x, b

In [102]:
# The problem wrapped into a class
class FEMPoissonProblem:
    def __init__(self, i):
        self.i = i
        number_of_points = 2**i + 1
        t = np.linspace(0, 1, number_of_points)
        T = utils.system_matrix_1d(i)
        b = f(t[1:-1])
        self.T = T
        self.b = b
        self.grid = t
        
    def solve(self):
        x = np.linalg.solve(self.T, self.b)
        return x, self.b

In [178]:
# Multigrid V-cycle
def MGV(b_i, x_i, i):
    """Recursive Multigrid V-cycle algorithm

    Parameters
    ----------
    b_i : numpy.array
        right-hand side of problem P(i), fine grid
        
    x_i : numpy.array
        solution of problem P(i), fine grid

    Returns
    -------
    numpy.array
        right-hand side of P(i)  
    numpy.array
        solution to the problem P(i)
    """
    if i == 1:
        T, b, t = P(1)
        x = b/T
        return b, x
    else:
        print("MGV called with i =", i)
        T_i, _, t_i = P(i)
        x_i = smooth(x=x_i, A=T_i, b=b_i)
        print('x_i after smoothing', x_i)
        t_i_r, x_i_r = restrict(t=t_i, x=x_i)
        print('restricted grid', t_i_r)
        print('restricted x', x_i_r)
        b_i_r = f(t_i_r)
        b_i_1, x_i_1 = MGV(b_i_r, x_i_r, i-1)

        t_i, d_i = interpolate(t_i_1, x_mgv)
        b_i = f(t_i)
        x_i = x_i - d_i
        x_i = smooth(x=x_i, A=T_i, b=b_i)
        return b_i, x_i
    

# Full Multigrid
def FMG(b_m, x_m, m):
    x_i_1, b_i_1  = solve(1)
    xs = []
    bs = []
    xs.append(x_i_1)
    bs.append(b_i_1)
    for i in range(2, m):
        b_In, x_In = interpolate(b_i_1, x_i_1)
        x_i, b_i = MGV(b_In, x_In, i)
        xs.append(x_i)
        bs.append(b_i)

In [179]:
i = 5
T, b, t = P(i)
x = np.linalg.solve(T, b)
x

array([0.01513672, 0.02929687, 0.04248047, 0.0546875 , 0.06591797,
       0.07617187, 0.08544922, 0.09375   , 0.10107422, 0.10742187,
       0.11279297, 0.1171875 , 0.12060547, 0.12304687, 0.12451172,
       0.125     , 0.12451172, 0.12304687, 0.12060547, 0.1171875 ,
       0.11279297, 0.10742187, 0.10107422, 0.09375   , 0.08544922,
       0.07617187, 0.06591797, 0.0546875 , 0.04248047, 0.02929687,
       0.01513672])

In [180]:
MGV(b_i=b, x_i=x, i=i)

MGV called with i = 5
x_i after smoothing [0.01513672 0.02929687 0.04248047 0.0546875  0.06591797 0.07617187
 0.08544922 0.09375    0.10107422 0.10742187 0.11279297 0.1171875
 0.12060547 0.12304687 0.12451172 0.125      0.12451172 0.12304687
 0.12060547 0.1171875  0.11279297 0.10742187 0.10107422 0.09375
 0.08544922 0.07617187 0.06591797 0.0546875  0.04248047 0.02929687
 0.01513672]
restricted grid [0.03125 0.09375 0.15625 0.21875 0.28125 0.34375 0.40625 0.46875 0.53125
 0.59375 0.65625 0.71875 0.78125 0.84375 0.90625]
restricted x [0.02905273 0.05444336 0.07592773 0.09350586 0.10717773 0.11694336
 0.12280273 0.12475586 0.12280273 0.11694336 0.10717773 0.09350586
 0.07592773 0.05444336 0.02905273]
MGV called with i = 4
x_i after smoothing [0.02913411 0.05444336 0.07592773 0.09350586 0.10717773 0.11694336
 0.12280273 0.12475586 0.12280273 0.11694336 0.10717773 0.09350586
 0.07592773 0.05444336 0.02913411]
restricted grid [0.0625 0.1875 0.3125 0.4375 0.5625 0.6875 0.8125]
restricted x [0

NameError: name 't_i_1' is not defined

In [143]:
#%run -i "test.py"§